In [3]:
import tkinter # representing gui 
from tkinter import*
from tkinter import ttk
import wx  #heigh quality gui
from summa.summarizer import summarize #summarize pdf
import PyPDF2 # reading pdf
import pyttsx3 # convert text to speach
import nltk
#nltk.download('punkt')
#from nltk.tokenize import sent_tokenize
#from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
import yake #keyword extraction 
import spacy #
import seaborn as sns
from rake_nltk import Rake
import re
import time
import threading
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg,NavigationToolbar2Tk)
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import  pyautogui
import os
import requests
import tkinter
from urllib.parse import urljoin
from bs4 import BeautifulSoup # for web scrawling
from tkinter import *
import pyautogui #for determine size of screen
from PIL import ImageTk, Image # for loading photo to our gui
#load gui content
def load(TotalPageNumber_entry,PageNumber_entry,VoiceType_entry,Rate_entry,volume_entry,ratio,Rvar):
    TotalPageNumber_entry=TotalPageNumber_entry
    PageNumber_entry=PageNumber_entry
    VoiceType_entry=VoiceType_entry
    Rate_entry=Rate_entry
    volume_entry=volume_entry
    ratio=ratio
    Rvar=Rvar
    print(TotalPageNumber_entry,PageNumber_entry,VoiceType_entry,Rate_entry,volume_entry)
    
    frm=tkinter.Tk()
    w=1600
    h=900


    text=''
# clear texts boxes from  current pdf 
    def clear():
        global text
        text=''
        Txt.delete('1.0', END)
        summarys.delete('1.0', END)
        keyw.delete('1.0', END)
       # text='not found'




    def show_keyw(x):
        x = set(x)
        x = list(x)

        for i in range(len (x)):
            keyw.insert(tkinter.END,str(x[i])+'\n'+'\n')
#extract most relevance keyword and  generate three type of graph
    def keywords(text):
        analytic_data=ttk.Label(frm,text='text analytic data',font=('tahoma bold',20),foreground='#116EAF')
        li=[]
        ki=[]
        minimum_numb=[]
        words_index=[]
        min_words=[]
        kw_extractor = yake.KeywordExtractor()
        keywords = kw_extractor.extract_keywords(text) 
        for kw ,vk in keywords:
            li.append(kw)
            ki.append(vk)
        s1 = set(li)
        s2 = set(ki)
        s1 = list(s1)
        s2 = list(s2)
        fig = Figure(figsize=(12,5)) # create a figure object
        ax = fig.add_subplot(111) # add an Axes to the figure
        for i in range(6):
            minimum_numb.append(min(s2))
            words_index.append(s2.index(min(s2)))
            s2.remove(min(s2))
        for j in words_index:
            min_words.append(s1[j])  
            
        show_keyw(min_words)
        if Rvar==1:
            ax.plot(min_words,minimum_numb)
        elif Rvar==2:
            ax.bar(min_words,minimum_numb)
        elif Rvar==3:
            
            wedges, plt_labels, junk =ax.pie(minimum_numb,labels=min_words,autopct='%.1f%%',explode=(.1,.1,.1,.1,.1,.1),shadow=True,radius = 1.3)
            ax.legend(wedges, min_words, loc = "upper right",bbox_to_anchor =(2, 1.1))

        chart1 = FigureCanvasTkAgg(fig,frm)
        chart1.get_tk_widget().place(x=w*1/2.5, y=h*.50)
        analytic_data.place(x=w*1/1.70, y=h*.45)
        keyw.place(x=w*1/1.3, y=h*.06)
        keyw_lbl.place(x=w*1/1.26, y=h*.01)
        
        
        
        
    
    def summary(text,ratio):
        summarys.insert(tkinter.END, summarize(text,ratio=ratio))


#convert text to speech and automaticlspeek with female sound or male sound
    def say(text):


        speak = pyttsx3.init()

        voices = speak.getProperty('voices')
        speak.setProperty('voice', voices[VoiceType_entry].id)  #Female,male
        speak.setProperty('rate', Rate_entry)
        speak.setProperty('volume',volume_entry) 

        speak.say(text)
        speak.runAndWait()

#know how many pages in pdf seleted
    def readfile(path):
        global text
        path = open(str(path),'rb')
        pdfReader = PyPDF2.PdfFileReader(path)
        totalpages = pdfReader.numPages
        text=''
        
        if TotalPageNumber_entry==0:

            for i in range(totalpages):
                from_page = pdfReader.getPage(i)
                text =text+from_page.extractText()
                print(text)
                Txt.insert(tkinter.END, text)
            
        elif TotalPageNumber_entry==1:
            from_page = pdfReader.getPage(PageNumber_entry)
            text =text+from_page.extractText()
            print(text)
            Txt.insert(tkinter.END, text)
        else:
            for i in range(TotalPageNumber_entry):
                from_page = pdfReader.getPage(i)
                text =text+from_page.extractText()
                print(text)
                Txt.insert(tkinter.END, text)

        SayBtn=Button(frm,text='Say',font=('Helvetica 20 bold'), bg='#1592B7',  fg = '#FFFFFF', height = 1, width = 5,command = lambda:say(text))
        ClearBtn.place(x=w*1/3.5, y=h*.82)
        SayBtn.place(x=w*1/5, y=h*.82)
        summarys.place(x=w*1/2.5, y=h*.06)
        sum_lbl.grid(row=0,column=2,padx=220)
        summary(text,ratio)
        keywords(text)
        
        
        
        

            
            
        
        
       
    #get pdf from  folder onn your pc 

    def get_path(wildcard):
        app = wx.App(None)
        style = wx.FD_OPEN | wx.FD_FILE_MUST_EXIST
        dialog = wx.FileDialog(None, 'Open', wildcard=wildcard, style=style)
        if dialog.ShowModal() == wx.ID_OK:
            path = dialog.GetPath()
        else:
            path = None
        dialog.Destroy()
        readfile(path)

                   
        
    sw=frm.winfo_screenwidth()
    sh=frm.winfo_screenheight()
  
    w,h = pyautogui.size()
    frm.geometry('%dx%d'%(w,h))

    title=ttk.Label(frm,text='read pdf',font=('tahoma bold',20),foreground='#116EAF')

    Txt = Text(fw     rm, height = 35,
                    width = 60,
                    font=('Helvetica 12 bold'),bg='#06395C',fg='#FFFFFF')


    Scrollbar=tkinter.Scrollbar(frm, orient=VERTICAL)
    Scrollbar.config(command=Txt.yview)
    Txt.config(yscrollcommand=Scrollbar.set)
    sum_lbl=ttk.Label(frm,text='summarize',font=('tahoma bold',20),foreground='#116EAF')
    keyw_lbl=ttk.Label(frm,text='key words',font=('tahoma bold',20),foreground='#116EAF')
    summarys=Text(frm, height = 20,width = 70,font=('Helvetica 10 bold'),bg='#06395C',fg='#FFFFFF')
    keyw=Text(frm, height = 17,width = 30,font=('Helvetica 12 bold'))


    
    SelectFileBtn=Button(frm,text='select pdf file',font=('Helvetica 20 bold'), bg='#1592B7',fg ='#FFFFFF', height = 1, width = 13,command=lambda:get_path('*.pdf'))
    ClearBtn=Button(frm,text='clear',font=('Helvetica 20 bold'), bg='#1592B7',fg ='#FFFFFF', height = 1, width = 5,command=clear)

    title.grid(row=0,column=0,pady=10)
    Txt.grid(row=1,column=0,padx=20)
    #SelectFileBtn.grid(row=2,column=0,pady=10)
    SelectFileBtn.place(x=w*1/60, y=h*.82)
    Scrollbar.grid(column=1, row=0, rowspan=2,sticky=NS)

    frm.mainloop()

  

In [4]:
fram=tkinter.Tk()
from PIL import ImageTk, Image
import  pyautogui
#to ignore total number of pages ,page number
def analysePdf():
    TotalPageNumber_entry.config(state= "normal")
    TotalPageNumber_entry.delete(0, END)
    TotalPageNumber_entry.insert(0,'0')
    TotalPageNumber_entry.config(state= "disabled")
    
    PageNumber_entry.config(state= "normal")
    PageNumber_entry.delete(0, END)
    PageNumber_entry.insert(0,'0')
    PageNumber_entry.config(state= "disabled")
    #to ignore total number of pages 
def analysOnePage() :
    TotalPageNumber_entry.config(state= "normal")
    TotalPageNumber_entry.delete(0, END)
    TotalPageNumber_entry.insert(0,'1')
    TotalPageNumber_entry.config(state= "disabled")
    
    PageNumber_entry.config(state= "normal")
    PageNumber_entry.delete(0, END)
    
    
    #ignore page number
    
def analysSomePages():
    PageNumber_entry.config(state= "normal")
    PageNumber_entry.delete(0, END)
    PageNumber_entry.insert(0,'0')
    PageNumber_entry.config(state= "disabled")
    
    TotalPageNumber_entry.config(state= "normal")
    TotalPageNumber_entry.delete(0, END)
    
    
def clear_entry():
    PageNumber_entry.config(state= "normal")
    TotalPageNumber_entry.config(state='normal')
    TotalPageNumber_entry.delete(0, 'end')
    PageNumber_entry.delete(0, 'end')
    # to extract pdf links from pages and download it on your pc
def web_scrapping():

    root = Tk()
    root.title("Welcome to GeekForGeeks")
    w, h = pyautogui.size()
    root.minsize(w,h)
    txt = Entry(root, width=73)
    label1=Label(root,text="Enter URL",font=("Helvetica", 16),foreground='#116EAF')
    label1.place(x=680,y=240)
    txt.place(x=480,y=270)
    #for logo
    img  = Image.open("chernobys_logo.png")
    img = img.resize((600, 150), Image.ANTIALIAS)
    photo=ImageTk.PhotoImage(img)
    lab=Label(image=photo).grid(row=0,column=0,columnspan=3,pady=20)
    #actioned when click button
    def clicked():
        url = txt.get()
    #If there is no such folder, the script will create one automatically
        folder_location = r'E:\webscraping'
        if not os.path.exists(folder_location):os.mkdir(folder_location)

        response = requests.get(url)
        soup= BeautifulSoup(response.text, "html.parser")     
        for link in soup.select("a[href$='.pdf']"):
        #Name the pdf files using the last portion of each link which are unique in this case
            filename = os.path.join(folder_location,link['href'].split('/')[-1])
            with open(filename, 'wb') as f:
                f.write(requests.get(urljoin(url,link['href'])).content)


    # inside
    btn = Button(root, text = "Next" ,bg="#116EAF"
                 ,fg = "white", command=clicked,width=40,height=1,padx=20,pady=20,font=("Helvetica", 16))
    btn.place(x=430, y=600)

    root.mainloop()
    
    
    
    

w,h = pyautogui.size()
sw=fram.winfo_screenwidth()
sh=fram.winfo_screenheight()
x=(sw-w)/2
y=(sh-h)/2
fram.geometry('%dx%d+%d+%d'%(w,h,x,y))


img  = Image.open("chernobys_logo.png")
img = img.resize((500, 150), Image.ANTIALIAS)
photo=ImageTk.PhotoImage(img)


TotalPageNumberVar=tkinter.StringVar()
PageNumberVar=tkinter.StringVar()
VoiceTypeVar=tkinter.StringVar()
RateVar=tkinter.StringVar()
volumeVar=tkinter.StringVar()
ratioVar=tkinter.StringVar()
Rvar = IntVar()


TotalPageNumber=ttk.Label(fram,text='tota pages',font=('tahoma bold',20),foreground='#116EAF')

VoiceType=ttk.Label(fram,text='voice type',font=('tahoma bold',20),foreground='#116EAF')
Rate=ttk.Label(fram,text='voice rate',font=('tahoma bold',20),foreground='#116EAF')
PageNumber=ttk.Label(fram,text='page number',font=('tahoma bold',20),foreground='#116EAF')
volume=ttk.Label(fram,text='voice volume',font=('tahoma bold',20),foreground='#116EAF')
ratio=ttk.Label(fram,text='summary ratio',font=('tahoma bold',20),foreground='#116EAF')

TotalPageNumber_entry=ttk.Entry(fram,font=('arial',15),textvariable=TotalPageNumberVar)
VoiceType_entry=ttk.Entry(fram,font=('arial',15),textvariable=VoiceTypeVar)
Rate_entry=ttk.Entry(fram,font=('arial',15),textvariable=RateVar)
PageNumber_entry=ttk.Entry(fram,font=('arial',15),textvariable=PageNumberVar)
volume_entry=ttk.Entry(fram,font=('arial',15),textvariable=volumeVar)
ratio_entry=ttk.Entry(fram,font=('arial',15),textvariable=ratioVar)


TotalPageNumber.grid(row=2,column=0,pady=20,padx=100)
VoiceType.grid(row=3,column=0,pady=20,padx=20)
Rate.grid(row=4,column=0,pady=20,padx=20)
PageNumber.grid(row=5,column=0,pady=20,padx=20)
volume.grid(row=6,column=0,pady=20,padx=20)

TotalPageNumber_entry.grid(row=2,column=1,pady=20,padx=150)
VoiceType_entry.grid(row=3,column=1,pady=20,padx=40)
Rate_entry.grid(row=4,column=1,pady=20,padx=40)
PageNumber_entry.grid(row=5,column=1,pady=20,padx=40)
volume_entry.grid(row=6,column=1,pady=20,padx=40)
ratio_entry.grid(row=7,column=1,pady=20,padx=40)



confirmBtn=Button(fram,text='confirm',font=('Helvetica 20 bold'), bg='#1592B7',fg ='#FFFFFF', height = 1, width = 18,command=lambda:load(int(str(TotalPageNumber_entry.get())),int(str(PageNumber_entry.get())),int(str(VoiceType_entry.get())),int(str(Rate_entry.get())),float(str(volume_entry.get())),float(str(ratio_entry.get())),Rvar.get()))
analysPdfBtn=Button(fram,text='analyse pdf',font=('Helvetica 20 bold'), bg='#1592B7',fg ='#FFFFFF', height = 1, width = 18,command=analysePdf)
analysOnePageBtn=Button(fram,text='analyse one page',font=('Helvetica 20 bold'), bg='#1592B7',fg ='#FFFFFF', height = 1, width = 18,command=analysOnePage)
analysSomePagesBtn=Button(fram,text='analyse some pages',font=('Helvetica 20 bold'), bg='#1592B7',fg ='#FFFFFF', height = 1, width = 18,command=analysSomePages)
plotRBtn = Radiobutton(fram, text="plot",font=('Helvetica 20 bold'), variable=Rvar, value=1)
barRBtn = Radiobutton(fram, text="bar",font=('Helvetica 20 bold'), variable=Rvar, value=2)
pieRBtn = Radiobutton(fram, text="pie",font=('Helvetica 20 bold'), variable=Rvar, value=3)
web_scrapping_btn=Button(fram,text='web scrapping',font=('Helvetica 20 bold'), bg='#1592B7',fg ='#FFFFFF', height = 1, width = 18,command=web_scrapping)


lab=Label(image=photo).grid(row=0,column=0,columnspan=3,pady=20)


analysPdfBtn.grid(row=3,column=4,padx=100)
analysOnePageBtn.grid(row=4,column=4,padx=100)
analysSomePagesBtn.grid(row=5,column=4,padx=100)
ratio.grid(row=7,column=0,pady=20)
plotRBtn.grid(row=3,column=3,padx=40)
barRBtn.grid(row=4,column=3,padx=40)
pieRBtn.grid(row=5,column=3,padx=40)
confirmBtn.grid(row=8,column=1,padx=20,pady=50)
web_scrapping_btn.grid(row=6,column=4)

fram.mainloop()


0 0 1 150 1.0
17
MICROCOMPUTER SOFTWARE DEVELOPMENT FACILITIES
J. S. Gorman and C. Mathiasen
Ford Aerospace . :
Houston, Texas
Historically, microcomputer software/firmware has been developed on a system designed to
perform software development and emulation functions for a specific processor. This
method proved successful during the early years of microcomputer software development.
However, both the number of microcomputers used in system design, and the complexity of
the microcomputer software, have increased. These changes dictate that more efficient and
cost effective methods be found for developing microcomputer software.
One approach is to utilize a host computer with high-speed peripheral support. Application
programs such as cross assemblers, loaders, and simulators are implemented in the host
computer for each of the microcomputers for which software development is a requirement.
The host computer is configured to operate in a time-share mode for multi-users. The
provided rem